<a href="https://colab.research.google.com/github/shakshi12/RumourGNN/blob/master/charliehebdo_gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install graphviz libgraphviz-dev libcgraph6

In [ ]:
!pip install spektral

In [ ]:
!git clone https://github.com/danielegrattarola/spektral.git

In [ ]:
cd spektral

In [ ]:
!python setup.py install

In [ ]:
from spektral.datasets import citation
data = citation.load_data('cora')
A, X, y, train_mask, val_mask, test_mask = data


N = A.shape[0]
F = X.shape[-1]
n_classes = y.shape[-1]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
pwd

In [ ]:
import io
import pandas as pd
df = pd.read_csv('/content/spektral/charliehebdo.csv', header = None)
#df = pd.read_csv(io.BytesIO(uploaded['dump_charliehebdo.csv']), header = None)
#df.to_csv('/content/drive/My Drive/charliehebdo.csv')
df = df.loc[:,[0,1,5,6,10]]
df.columns = ['src_id','src_text','reply_id','reply_text','labels']
y = df['labels']
X = pd.read_csv('/content/drive/My Drive/X.csv')
from scipy import sparse
X_csr = sparse.csr_matrix(X)

In [ ]:

X1 = X[:21073]

In [ ]:
X1.shape

In [ ]:
X = X1

In [ ]:
from scipy import sparse
X_csr = sparse.csr_matrix(X)

In [ ]:
import networkx as nx
short_graph = df
short_graph = short_graph[0:20000]
G = nx.from_pandas_edgelist(df=short_graph, source='src_id', target='reply_id')
nodes = list(short_graph.src_id)
nodes1 = list(short_graph.reply_id)
nodes.extend(nodes1)
G.add_nodes_from(nodes_for_adding=nodes)
A = nx.adjacency_matrix(G)

In [ ]:
A1 = A.todense()

In [ ]:
A.shape

In [ ]:
df1 = df[:21073]

In [ ]:
#from spektral.datasets import citation
#data = citation.load_data('cora')
#A, X, y, train_mask, val_mask, test_mask = data

X = X_csr
l = []
import numpy as np
for i in range(360):
  l.append(True)
for i in range (20713):
  l.append(False)
train_mask = np.array(l)
l1 = []
for i in range(360):
  l1.append(False)
for i in range(829):
  l1.append(True)
for i in range(19884):
  l1.append(False)
val_mask = np.array(l1)
l2 = []
for i in range(15884):
  l2.append(False)
for i in range(5189):
  l2.append(True)
test_mask = np.array(l2)
y = np.zeros((21073,2))
n1 = df1['labels']
n = list(n1)
for i in range(21073):
  if (n[i] == 1):
    y[i,1] = 1
  elif (n[i] == 0):
    y[i,0] = 1



N = A.shape[0]
F = X.shape[-1]
n_classes = y.shape[-1]

In [ ]:
from spektral.layers import GraphConv
from keras.models import Model
from keras.layers import Input, Dropout

In [ ]:
# Model definition
X_in = Input(shape=(F, ))  # Input layer for X
A_in = Input((N, ), sparse=True)  # Input layer for A

graph_conv_1 = GraphConv(16, activation='relu')([X_in, A_in])
dropout = Dropout(0.5)(graph_conv_1)
graph_conv_2 = GraphConv(n_classes, activation='softmax')([dropout, A_in])

# Build model
model = Model(inputs=[X_in, A_in], outputs=graph_conv_2)

In [ ]:
from spektral import utils
A = utils.localpooling_filter(A)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

In [ ]:
# Train model
validation_data = ([X, A], y, val_mask)
model.fit([X, A],
          y,
          sample_weight=train_mask,
          epochs=100,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False)  # Shuffling data means shuffling the whole graph

In [ ]:
# Evaluate model
eval_results = model.evaluate([X, A],
                              y,
                              sample_weight=test_mask,
                              batch_size=N)
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))